In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.insert(1, 'src/')
import utilities
import config
import csv

from fastai.text import *

In [2]:
# Load the data for LM finetuning

SARC_DATA = config.SARC_DATA
SARC_MAIN = config.SARC_MAIN
SARC_POL = config.SARC_POL

train_path = SARC_MAIN + 'train-unbalanced.csv'
test_path = SARC_MAIN + 'test-unbalanced.csv'
comment_path = SARC_MAIN + 'comments.json'

train_docs, test_docs, train_labels, test_labels = \
    utilities.load_sarc_responses(train_path, test_path, comment_path)

In [3]:
# Format data for LM finetuning
# TODO create funtion for this purpose

learn_data = []
invalids = ['n/a', 'N/A', 'null', 'NULL', 'na', 'NA', 'nan', 'NaN', 'NAN']

for index, sample in enumerate(train_labels):
    ancestor = train_docs['ancestors'][index][0]
    learn_data.append([-1, ancestor])
    for i, label in enumerate(sample):
        response = train_docs['responses'][index][i]
        if response not in invalids:
            learn_data.append([int(label), response])

for index, sample in enumerate(test_labels):
    ancestor = test_docs['ancestors'][index][0]
    learn_data.append([-1, ancestor])
    for i, label in enumerate(sample):
        response = test_docs['responses'][index][i]
        if response not in invalids:
            learn_data.append([int(label), response])

In [4]:
# Write csv with LM finetuning data

with open('DATA/finetune_data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(learn_data)

In [5]:
# Load databunch for lm training
ml_bunch = TextLMDataBunch.from_csv(path='.', csv_name='DATA/finetune_data.csv')

# Save databunch
ml_bunch.save('DATA/lm_databunch')

In [3]:
# Load the data for classification

SARC_DATA = config.SARC_DATA
SARC_MAIN = config.SARC_MAIN
SARC_POL = config.SARC_POL

train_path = SARC_MAIN + 'train-balanced.csv'
test_path = SARC_MAIN + 'test-balanced.csv'
comment_path = SARC_MAIN + 'comments.json'

train_docs, test_docs, train_labels, test_labels = \
    utilities.load_sarc_responses(train_path, test_path, comment_path)

In [4]:
# Format data for classification

learn_data = []
invalids = ['n/a', 'N/A', 'null', 'NULL', 'na', 'NA', 'nan', 'NaN', 'NAN']

for index, sample in enumerate(train_labels):
    ancestor = train_docs['ancestors'][index][0]
    for i, label in enumerate(sample):
        response = train_docs['responses'][index][i]
        if response not in invalids:
            learn_data.append([int(label), ancestor + ' ^ ' + response])

In [5]:
# Write csv with classification data

with open('DATA/class_data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(learn_data)

In [6]:
# load databunch for classification training
class_bunch = TextClasDataBunch.from_csv(path='.', csv_name='DATA/class_data.csv')

# save databunch
class_bunch.save('DATA/class_databunch')